## Check the manual QC of ANMN against automated. 


In [4]:
!pip install kerchunk

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 41 kB 162 kB/s eta 0:00:011
     |████████████████████████████████| 45 kB 1.1 MB/s eta 0:00:011


In [4]:
import fsspec
from fsspec.core import url_to_fs
fs = fsspec.filesystem('s3',use_listings_cache=False,anon=True,)

urls = fs.glob('s3::imos-data/IMOS/ANMN/NSW/CH100/Temperature/*.nc')
urls[0:5]

['imos-data/IMOS/ANMN/NSW/CH100/Temperature/IMOS_ANMN-NSW_TZ_20090812T020004Z_CH100_FV01_CH100-0908-Aqualogger-520PT-99.5_END-20091001T145004Z_C-20160824T072817Z.nc',
 'imos-data/IMOS/ANMN/NSW/CH100/Temperature/IMOS_ANMN-NSW_TZ_20091214T020000Z_CH100_FV01_CH100-0912-Aqualogger-520T-28.5_END-20100226T023000Z_C-20160824T073255Z.nc',
 'imos-data/IMOS/ANMN/NSW/CH100/Temperature/IMOS_ANMN-NSW_TZ_20091214T020000Z_CH100_FV01_CH100-0912-Aqualogger-520T-36.5_END-20100226T020000Z_C-20160824T073254Z.nc',
 'imos-data/IMOS/ANMN/NSW/CH100/Temperature/IMOS_ANMN-NSW_TZ_20091214T020000Z_CH100_FV01_CH100-0912-Aqualogger-520T-44.5_END-20100226T013000Z_C-20160824T073254Z.nc',
 'imos-data/IMOS/ANMN/NSW/CH100/Temperature/IMOS_ANMN-NSW_TZ_20091214T020000Z_CH100_FV01_CH100-0912-Aqualogger-520T-60.5_END-20100226T005500Z_C-20160824T073254Z.nc']

In [2]:
from dask.distributed import Client
client = Client(n_workers=16,threads_per_worker=16) 
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/ribeiron/proxy/8787/status,
Dashboard: /user/ribeiron/proxy/8787/status,Workers: 16
Total threads: 256,Total memory: 8.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37769,Workers: 16
Dashboard: /user/ribeiron/proxy/8787/status,Total threads: 256
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:40769,Total threads: 16
Dashboard: /user/ribeiron/proxy/33349/status,Memory: 512.00 MiB
Nanny: tcp://127.0.0.1:46261,


In [ ]:
import s3fs
import xarray as xr
from dask import bag as db

def open_cdt(url):
    s3 = s3fs.S3FileSystem(anon=True,default_fill_cache=False,default_cache_type=None)
    with s3.open(url,) as f:
        data=xr.open_dataset(f,engine='h5netcdf').load()
    return data


b = db.from_sequence(ref_files[0:10])
cast = db.map(open_cdt,b).compute()

In [8]:
import kerchunk.hdf
import fsspec


so = dict(
    anon=True, default_fill_cache=False, default_cache_type='first'
)
singles = []
for u in urls:
    print(u)
    with fsspec.open('s3:://'+u, **so) as inf:
        h5chunks = kerchunk.hdf.SingleHdf5ToZarr(inf, u, inline_threshold=100)
        singles.append(h5chunks.translate())

imos-data/IMOS/ANMN/NSW/CH100/Temperature/IMOS_ANMN-NSW_TZ_20090812T020004Z_CH100_FV01_CH100-0908-Aqualogger-520PT-99.5_END-20091001T145004Z_C-20160824T072817Z.nc


FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/s3'

In [ ]:
manual_qc = list(filter(lambda x: 'manually flagged' in x.attrs['quality_control_log'], cast))


In [18]:
manual_qc

[]